In [8]:
import pandas as pd
import re
from datetime import datetime
# --- FUNGSI BARU UNTUK MENERJEMAHKAN WAKTU ---
def parse_relative_time(relative_str):
    """
    Fungsi untuk mengubah string waktu relatif (misal: "2 bulan lalu")
    menjadi objek datetime yang sebenarnya.
    """
    if pd.isna(relative_str) or not isinstance(relative_str, str):
        return pd.NaT

    relative_str = str(relative_str).lower()
    now = pd.Timestamp.now()
    num = 1 # Default untuk kasus "sebulan", "setahun"
    
    # Cari angka dalam string
    num_match = re.search(r'\d+', relative_str)
    if num_match:
        num = int(num_match.group(0))

    # Tentukan unit waktu dan hitung tanggalnya
    if 'hari' in relative_str:
        delta = pd.Timedelta(days=num)
    elif 'minggu' in relative_str:
        delta = pd.Timedelta(weeks=num)
    elif 'bulan' in relative_str:
        delta = pd.Timedelta(days=num * 30) # Menggunakan pendekatan 30 hari/bulan
    elif 'tahun' in relative_str:
        delta = pd.Timedelta(days=num * 365) # Menggunakan pendekatan 365 hari/tahun
    else:
        return pd.NaT # Kembalikan NaT jika format tidak dikenali
        
    return now - delta

# Ganti dengan path file Excel Anda yang sebenarnya
df_destinasi_raw = pd.read_excel('./data/tempat wisata sumedang.xlsx')
df_review_raw = pd.read_excel('./data/sumedang reviews.xlsx')

# --- Membersihkan Data Destinasi ---
df_destinasi = df_destinasi_raw.rename(columns={
    'NAMA': 'name',
    'KATEGORI': 'category_str',
    'LATITUDE': 'latitude',
    'LONGITUDE': 'longitude',
    'ALAMAT LENGKAP': 'address',
    'DESKRIPSI SINGKAT': 'description'
})
df_destinasi = df_destinasi[['name', 'category_str', 'latitude', 'longitude', 'address', 'description']]


# --- Membersihkan Data Review (dengan fungsi baru) ---
df_review = df_review_raw.rename(columns={
    'place': 'destination_name',
    'user': 'user_name',
    'review': 'review_text',
    'rating': 'rating',
    'time': 'review_time'
})
df_review = df_review[['destination_name', 'user_name', 'review_text', 'rating', 'review_time']]

df_review['rating'] = pd.to_numeric(df_review['rating'], errors='coerce')
df_review = df_review.dropna(subset=['rating', 'destination_name'])

# --- PERBAIKAN DI SINI ---
# Gunakan fungsi 'parse_relative_time' yang baru kita buat
df_review['review_time'] = df_review['review_time'].apply(parse_relative_time)


print("Data Review Setelah Dibersihkan (waktu relatif sudah diubah menjadi tanggal):")
display(df_review.head())

# Tampilkan beberapa baris acak untuk memastikan konversi berhasil
print("\nContoh hasil konversi waktu (sampel acak):")
display(df_review.sample(5))

Data Review Setelah Dibersihkan (waktu relatif sudah diubah menjadi tanggal):


,destination_name,user_name,review_text,rating,review_time
0,95 Farm villa resto,Puspito Lie Yin,"Mantappp semua.\nSuasana sangat sejuk, view ba...",5.0,2025-09-03 03:57:23.447727
1,95 Farm villa resto,Budi,"Tempat nyaman, masing2 saung ada karaokenya, p...",5.0,2025-07-05 03:57:23.452499
2,95 Farm villa resto,kaka darsuki,Ada di sumedang tempat dingin yg full makanan ...,5.0,2023-10-04 03:57:23.452499
3,95 Farm villa resto,Nurlaila Ramdayani,"Makanannya enak, masakan sunda. Suasananya ena...",5.0,2024-10-03 03:57:23.452499
4,95 Farm villa resto,SiNyoman AWww,"Tempat nya bagus, makanan standar rasa enak, s...",4.0,2023-10-04 03:57:23.452499



Contoh hasil konversi waktu (sampel acak):


,destination_name,user_name,review_text,rating,review_time
27860,Tahura Gunung Kunci,lalal lulul,Gunung Tahura berada dekat dengan Alun2 kota S...,5.0,2020-10-04 03:57:24.602025
32201,Tugu Buah Mangga,Azril Khaerul putra,(No Review),5.0,2024-10-03 03:57:24.685953
25941,Saung Cibingbin Sumedang,Riyan Wisnu Sunarya,"Penuh sekali, klo kesini mending pake motor, k...",5.0,2022-10-04 03:57:24.566595
12891,Kiarapayung Camp,Hoeda Aulia,"suasana sejuk, cocok buat olahraga jogging ata...",5.0,2025-03-07 03:57:24.176489
31461,Tugu Binokasih,fauzi rahman,Sumedang banget,5.0,2024-10-03 03:57:24.671552


In [9]:
# --- Destinasi dan Kategori (kode ini sama seperti sebelumnya) ---
df_destinasi['destination_id'] = df_destinasi.index + 1
df_destinasi['category_list'] = df_destinasi['category_str'].str.split(',').apply(lambda x: [i.strip() for i in x])
all_categories = set([cat for sublist in df_destinasi['category_list'] for cat in sublist])
df_categories = pd.DataFrame(list(all_categories), columns=['category_name'])
df_categories['category_id'] = df_categories.index + 1
dest_cat_relations = []
for _, row in df_destinasi.iterrows():
    for cat_name in row['category_list']:
        cat_id = df_categories[df_categories['category_name'] == cat_name]['category_id'].iloc[0]
        dest_cat_relations.append({'destination_id': row['destination_id'], 'category_id': cat_id})
df_destination_categories = pd.DataFrame(dest_cat_relations)

# --- Pengguna dan Rating (dengan 'review_time') ---
all_users = df_review['user_name'].unique()
df_users = pd.DataFrame(all_users, columns=['user_name'])
df_users['user_id'] = df_users.index + 1

dest_name_to_id = df_destinasi.set_index('name')['destination_id']
user_name_to_id = df_users.set_index('user_name')['user_id']

df_review['destination_id'] = df_review['destination_name'].map(dest_name_to_id)
df_review['user_id'] = df_review['user_name'].map(user_name_to_id)

# Buat DataFrame final untuk rating, sekarang termasuk 'review_time'
df_ratings = df_review[['user_id', 'destination_id', 'rating', 'review_text', 'review_time']].dropna()
df_ratings[['user_id', 'destination_id']] = df_ratings[['user_id', 'destination_id']].astype(int)

# --- Simpan semua ke CSV ---
df_destinasi[['destination_id', 'name', 'description', 'latitude', 'longitude', 'address']].to_csv('destinations.csv', index=False)
df_categories.to_csv('categories.csv', index=False)
df_destination_categories.to_csv('destination_categories.csv', index=False)
df_users.to_csv('users.csv', index=False)
df_ratings.to_csv('ratings.csv', index=False) # File ini sekarang punya kolom 'review_time'

print("Semua data berhasil disimpan ke file CSV!")
print("\nContoh data ratings.csv:")
display(df_ratings.head())

Semua data berhasil disimpan ke file CSV!

Contoh data ratings.csv:


,user_id,destination_id,rating,review_text,review_time
0,1,1,5.0,"Mantappp semua.\nSuasana sangat sejuk, view ba...",2025-09-03 03:57:23.447727
1,2,1,5.0,"Tempat nyaman, masing2 saung ada karaokenya, p...",2025-07-05 03:57:23.452499
2,3,1,5.0,Ada di sumedang tempat dingin yg full makanan ...,2023-10-04 03:57:23.452499
3,4,1,5.0,"Makanannya enak, masakan sunda. Suasananya ena...",2024-10-03 03:57:23.452499
4,5,1,4.0,"Tempat nya bagus, makanan standar rasa enak, s...",2023-10-04 03:57:23.452499
